In [1]:
import winsound as ws
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import cv2
from sklearn.cluster import KMeans
from multiprocessing import Pool

<h1>Ratings - Users merging</h1>
<p>This code merge information about Users and their ratings</p>

In [2]:
userbook = pd.read_csv("dataset\\ratings.csv")
userbook.head()
#remember: book_id equals id in books.csv

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [3]:
complessivo = pd.read_csv("dataset\\finals\\complissivolibricongenere.csv")
complessivo = complessivo.drop(axis = 1, columns=["Unnamed: 0"])
complessivo

,id,book_id,authors,title,original_publication_year,average_rating,work_ratings_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,color_name,price,color,pricetag,genderid,gendernames,genderclass,discreteauthors
0,1,2767052,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",2008.0,4.34,4942365,0.013499,0.025886,0.113325,0.299716,0.547575,Black,6.83,0,2,11305.0,fantasy,4,181
1,2,3,J.K. Rowling,Harry Potter and the Sorcerer's Stone (Harry P...,1997.0,4.44,4800065,0.015730,0.021182,0.094795,0.240896,0.627396,White,49.78,1,4,11305.0,fantasy,4,40
2,3,41865,Stephenie Meyer,"Twilight (Twilight, #1)",2005.0,3.57,3916824,0.116470,0.111519,0.202541,0.223414,0.346056,Black,16.99,0,3,31745.0,vampires,6,104
3,4,2657,Harper Lee,To Kill a Mockingbird,1960.0,4.25,3340896,0.018087,0.035145,0.133747,0.299905,0.513116,Blue,16.99,4,3,18367.0,literature,10,1074
4,5,4671,F. Scott Fitzgerald,The Great Gatsby,1925.0,3.89,2773745,0.031090,0.071247,0.218534,0.337454,0.341675,Brown,17.00,8,3,18367.0,literature,10,334
5,6,11870085,John Green,The Fault in Our Stars,2012.0,4.26,2478609,0.019363,0.037409,0.132151,0.281800,0.529277,Cyan,19.99,7,3,26138.0,romance,11,355
6,7,5907,J.R.R. Tolkien,The Hobbit,1937.0,4.25,2196809,0.020950,0.034953,0.131395,0.303001,0.509702,Black,10.99,0,2,11305.0,fantasy,4,192
7,8,5107,J.D. Salinger,The Catcher in the Rye,1951.0,3.79,2120637,0.051580,0.087483,0.214578,0.311942,0.334416,Orange,15.00,9,3,33114.0,young adult,0,490
8,9,960,Dan Brown,"Angels & Demons (Robert Langdon, #1)",2000.0,3.85,2078754,0.037446,0.070109,0.220531,0.344711,0.327203,Blue,12.37,4,2,20939.0,mystery,14,249
9,10,1885,Jane Austen,Pride and Prejudice,1813.0,4.24,2191465,0.024960,0.039464,0.129982,0.278241,0.527352,White,7.52,1,2,14487.0,historical fiction,9,222


In [4]:
userbook["book_id"].max() #proves that book_id in ratings.csv is id in books.csv!!!!!!!!!!!!!!

10000

In [5]:
#only positive reviews
positives = userbook.where(userbook["rating"] >= 4).dropna()
negatives = userbook.where(userbook["rating"] < 4).dropna()

In [6]:
#merge the informations aboute the books
list_id = complessivo["id"].values.tolist()
genders = complessivo["genderclass"].values.tolist()
prices = complessivo["pricetag"].values.tolist()
colours = complessivo["color"].values.tolist()
avgratings = complessivo["average_rating"].values.tolist()
list_genderclass = []
for i in range(len(list_id)):
    t = userbook[userbook.book_id == list_id[i]]
    t = t.assign(genderclass = lambda x: genders[i])
    t = t.assign(priceclass = lambda x: prices[i])
    t = t.assign(colorclass = lambda x: colours[i])
    t = t.assign(avgrating = lambda x: avgratings[i])
    list_genderclass.append(t)
userbook = pd.concat(list_genderclass)




In [7]:
#select only users that realy reads
axes = userbook["user_id"].value_counts().axes[0].tolist()
values = userbook["user_id"].value_counts().values.tolist()
tocatch =[]
for i in range(len(values)):
    if(values[i] >= 20):#trigger the number of books 
        tocatch.append(userbook[userbook.user_id == axes[i]])
    
userbook = pd.concat(tocatch, ignore_index =True)

In [8]:
userbook.head()

,book_id,user_id,rating,genderclass,priceclass,colorclass,avgrating
0,1,52036,2,4,2,0,4.34
1,2,52036,3,4,4,1,4.44
2,3,52036,1,6,3,0,3.57
3,4,52036,3,10,3,4,4.25
4,5,52036,4,10,3,8,3.89


In [9]:
#set the rating difference 
userbook = userbook.assign(differencerating = abs(userbook["rating"]-userbook["avgrating"]))

In [10]:
#users classifications
userbook.sort_values(["user_id"],ascending= True )
userslist = userbook["user_id"].drop_duplicates(keep="first").values.tolist()
users = pd.DataFrame(columns = ["user_id", "ratingpoint", "maincolor", "mainprice", "maingender"])
listnewusers = []
for user in userslist:
    usertable = userbook[userbook.user_id == user]
    point = sum(usertable["differencerating"]*usertable["differencerating"])/len(usertable["differencerating"])
    liked = usertable[usertable.rating >=4]
    if(not liked.empty):
        gender = liked["genderclass"].value_counts(sort=True, ascending=False).index[0]
    price = usertable["priceclass"].value_counts(sort=True, ascending=False).index[0]
    color = usertable["colorclass"].value_counts(sort=True, ascending=False).index[0]
    newuser = pd.DataFrame(data = {"user_id": [user], "ratingpoint": [point], "maincolor": [color], "mainprice": [price], "maingender": [gender]})
    listnewusers.append(newuser)
users = pd.concat(listnewusers, ignore_index=True)    

In [14]:
users.to_csv("dataset\\finals\\userresume.csv")
users

,maincolor,maingender,mainprice,ratingpoint,user_id
0,0,4,3,1.973407,52036
1,0,4,3,0.541586,28158
2,0,10,3,1.142220,12381
3,0,4,2,0.548976,30944
4,0,4,3,0.921369,12874
5,0,4,3,0.772097,6630
6,0,1,3,0.608944,45554
7,0,4,3,0.447080,19729
8,0,1,3,1.074634,7563
9,0,4,3,0.598650,37834
